<a href="https://colab.research.google.com/github/Alex-Steel-13/datascience/blob/main/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
###DO NOT EDIT THIS CODE
################################################################################################################################

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt
import os
import cv2
import time
import random
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.utils import make_grid
from torchvision.models import resnet50
from sklearn.model_selection import train_test_split
from PIL import Image
from torchvision.datasets import ImageFolder
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader, Dataset # Gives easier dataset managment and creates mini batches
from torchvision.datasets import ImageFolder
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
from PIL import Image

# GPUs are 3x faster than CPU. Better to use if it is available 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define Loss Function
loss_function = nn.CrossEntropyLoss()

# This function returns the number of parameters in the model
def num_params(model):
  return sum([p.numel() for p in model.parameters()])

# Define a Training Function. This function will: compute the forward pass, backpropagate,
# update the weights, and repeat the steps for a given number of epochs. At each epoch, 
# it will output the training loss and test loss at every step
def train(epochs, model, trainloader, testloader, optimizer, loss_function):
  for epoch in range(epochs):
    loss_epoch = np.array([])
    train_correct, train_total = 0, 0
    test_correct, test_total = 0, 0

    for data, labels in trainloader:
      # convert into GPU objects if needed
      input_data = data.to(device)
      labels = labels.to(device)

      # forward pass
      predict = model(input_data)
      
      # backward pass
      loss = loss_function(predict, labels)
      optimizer.zero_grad()
      loss.backward()

      # update parameters (weights and biases)
      optimizer.step()

      # store progress
      loss_epoch = np.append(loss_epoch, loss.item())

    # evaluate test accuracy
    for data, labels in testloader:
      input_data = data.to(device)
      labels = labels.to(device)
      predict = model(input_data)
      for i, out in enumerate(predict):
        pred = torch.argmax(out)
        if pred == labels[i]:
          test_correct+=1
        test_total+=1

    test_accuracy = test_correct/test_total    
  
    print('epoch [{}/{}], training loss:{:.4f}, test accuracy:{:.4f}'.format(epoch+1, epochs, np.mean(loss_epoch), test_accuracy))
################################################################################################################################

# **Load Dataset**

Available datasets are: MNIST, CIFAR10

## MNIST

In [24]:
# download and load data
batch_size = 512

# download and transform train dataset
train_loader = torch.utils.data.DataLoader(datasets.FashionMNIST('./mnist_data', download=True, train=True, transform=transforms.Compose([
                                                transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                ])), batch_size=batch_size, shuffle=True)

# download and transform test dataset
test_loader = torch.utils.data.DataLoader(datasets.FashionMNIST('./mnist_data', download=True, train=False, transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), batch_size=batch_size, shuffle=True)





In [25]:
# it is a good idea to take a look at the data. Here we see it is a 28x28 grayscale image
for data, labels in train_loader:
  print(data[0].size())
  break

torch.Size([1, 28, 28])


# **Build a Network and Define Hyperparameters**

In [26]:
########################
####                #### 
#### EDIT THIS CELL ####
####                ####
########################


learning_rate = 10e-3
weight_decay = 10e-5
n_epochs = 10

# neural network
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    ### Define Layers
    self.ll1 = nn.Linear(784,128)
    self.ll2 = nn.Linear(128,64)
    self.ll3 = nn.Linear(64,10)

  def forward(self, x):
    x = torch.flatten(x,1)
    x = self.ll1(x)
    x = torch.sigmoid(x)
    x = self.ll2(x)
    x = torch.sigmoid(x)
    x = self.ll3(x)
    x = torch.sigmoid(x)
    return x
  
# Every time you edit the neural network, you'll have to update this cell
# Create model object
model = NeuralNetwork().to(device)

# Loads Adam optimizer, which implements a version of gradient descent
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [27]:
# check the structure of your network
print(model)

# apply your model to a single input. This helps check that 
# the dimensions are correct
model(torch.rand(1,1,28,28, device=device))

NeuralNetwork(
  (ll1): Linear(in_features=784, out_features=128, bias=True)
  (ll2): Linear(in_features=128, out_features=64, bias=True)
  (ll3): Linear(in_features=64, out_features=10, bias=True)
)


tensor([[0.4925, 0.6005, 0.5146, 0.5620, 0.4622, 0.5990, 0.4565, 0.5052, 0.3870,
         0.4174]], device='cuda:0', grad_fn=<SigmoidBackward0>)

# **Train and Validate**

In [28]:
train(n_epochs, model, train_loader, test_loader, optimizer, loss_function)

epoch [1/10], training loss:1.7640, test accuracy:0.5411
epoch [2/10], training loss:1.6361, test accuracy:0.6639
epoch [3/10], training loss:1.6093, test accuracy:0.7582
epoch [4/10], training loss:1.5960, test accuracy:0.7783
epoch [5/10], training loss:1.5847, test accuracy:0.7842
epoch [6/10], training loss:1.5855, test accuracy:0.7799
epoch [7/10], training loss:1.5829, test accuracy:0.8110
epoch [8/10], training loss:1.5785, test accuracy:0.7956
epoch [9/10], training loss:1.5776, test accuracy:0.8256
epoch [10/10], training loss:1.5777, test accuracy:0.8170
